In [89]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.retrievers import AmazonKendraRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.agents import load_tools,Tool
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.agents import initialize_agent, AgentType


In [90]:
dynamodb_resource=boto3.resource('dynamodb')
boto3_bedrock = boto3.client("bedrock-runtime")
kendra_client=boto3.client(service_name='kendra')

In [98]:
#table_name_active_connections = os.environ.get('TABLE_CONNECTIONS')
table_name = "SessionTable"
table_name_agenda = "QaToAnAgendaStack-TblReInventAgendaB6AA89B7-1O9IOB868HBRH"
kendra_index_id = "04a17bad-8a17-4893-8f62-75341e5e789e"
lambda_query_function_name = "QaToAnAgendaStack-FndynamodbqueryC2D26368-RxuymCNMoCBm"
bedrock_embedding_model_id = 'amazon.titan-embed-text-v1'
model_id = "anthropic.claude-instant-v1"
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

In [99]:

llm = Bedrock(model_id=model_id, model_kwargs=model_parameter,client=boto3_bedrock)
tools = load_tools(
                        ["awslambda"],
                        awslambda_tool_name="search-session-id-information",
                        awslambda_tool_description="useful for searching session information data by their ID number, only send the ID number",
                        function_name=lambda_query_function_name,
                    )
#table_name = os.environ.get("TABLE_NAME")

In [100]:
def kendra_ask(kendra_index_id,llm):
    retriever = AmazonKendraRetriever(index_id=kendra_index_id)
    memory_kendra = ConversationBufferMemory(memory_key="chat_history", return_messages=True,ai_prefix="A",human_prefix="H")


    Kendra_prompt_template = """Human: You are an Agenda re:Invent 2023 Assistant. 
    You are talkative and provides specific details from its context.
    If the AI does not know the answer to a question, it truthfully says it 
    does not know.

    Assistant: OK, got it, I'll be a talkative truthful AI assistant.

    Human: Here are a few documents in <documents> tags:
    <documents>
    {context}
    </documents>
    Based on previous documents, provide a list of data for each documents (If you find more than one), example: spaker veneu, date... , it's okay if you respond with just one option, the information is important so I can decide the best choise for me to: {question}

    Assistant:
    """
    PROMPT = PromptTemplate(
        template=Kendra_prompt_template, input_variables=["context", "question"]
    )

    condense_qa_template_kendra = """{chat_history}
    Human:
    Given the previous conversation and a follow up question below, based on previous documents, provide a list of data for each document (If you find more than one), example: spaker veneu, date... , it's okay if you respond with just one option, the information is important so I can decide the best choise for me.
    Follow Up Question: {question}
    Standalone Question:

    Assistant:"""
    standalone_question_prompt_kendra = PromptTemplate.from_template(condense_qa_template_kendra)

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        retriever=retriever, 
        condense_question_prompt=standalone_question_prompt_kendra, 
        return_source_documents=False, 
        combine_docs_chain_kwargs={"prompt":PROMPT},
        memory = memory_kendra,
        verbose=True)

    return qa

In [101]:
def define_tools(qa,tools):
    tools.append(
    Tool.from_function(
        func=qa.run,
        name="re-invent-agenda-2023",
        description="useful when you want search sessions in re:Invent 2023 agenda. This will output documentation in text type, and /n, you must deliver a coherent and accurate response from all the documentation provided. ",
    )
    )
    return tools

In [102]:
query = "I'm looking for a session about generative ai"

kendra_ask(kendra_index_id,llm).run(query)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: You are an Agenda re:Invent 2023 Assistant. 
    You are talkative and provides specific details from its context.
    If the AI does not know the answer to a question, it truthfully says it 
    does not know.

    Assistant: OK, got it, I'll be a talkative truthful AI assistant.

    Human: Here are a few documents in <documents> tags:
    <documents>
    Document Title: BOA210_Generative_AI_Asking_for_a_friend.txt
Document Excerpt: 
November, 29 2023 16:30:00 -0500 locationname: South Seas F floorplanname: Level 3 | South venuename: Mandalay Bay sessiontype: Breakout Session description: As artificial intelligence permeates modern life, generative models are increasingly applied in various fields from art to science. However, many people still have basic questions or concerns about this fast-paced innovation. This session breaks down the complexities, starting with an eas

' Based on the documents provided, here is the information for sessions related to generative AI:\n\nDocument 1:\n- Speaker: Mike Chambers \n- Venue: Mandalay Bay\n- Date: November 29, 2023 16:30-17:30\n- Topic: Generative AI, providing an introduction and answering questions\n\nDocument 2: \n- Speakers: Shruti Diamant, Shruti Koparkar, Ion Stoica\n- Venue: Venetian \n- Date: November 28, 2023 23:30-00:30\n- Topic: Moving beyond hype and realizing value from generative AI, hearing from experts across industries\n\nBoth sessions seem relevant to learning about generative AI. The first document provides an introductory session on the topic, while the second has experts discussing real-world applications. Let me know if you need any other details to decide which session might better suit your interests and availability.'

In [103]:
def promp_definition():

    prompt_template = """
        You are an assistant who provides recommendations on re:invent 2023 sessions, you deliver the recommendation according to their question, and also do casual conversation. 
        Use the following format:
        History: the context of a previous conversation with the user. Useful if you need to recall past conversation, make a summary, or rephrase the answers. if History is empty it continues.
        Question: the input question you must answer
        Thought: you should always think about what to do, Also try to follow steps mentioned above.
        Action: the action to take, should be one of ['re-invent-agenda-2023',"search-session-id-information"], provides options that have the most information that can be useful in making a decision.
        Action Input: the input to the action
        Observation: the result of the action
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question, If you have more than one answer, give them all. Always reply in the original user language and human legible.

        History: 
        {chat_history}

        Question: {input}

        Assistant:
        {agent_scratchpad}"""

    updated_prompt = PromptTemplate(
    input_variables=['chat_history','input', 'agent_scratchpad'], template=prompt_template)

    return updated_prompt

In [104]:
def memory_dynamodb(id,table_name_session,llm):
    message_history = DynamoDBChatMessageHistory(table_name=table_name_session, session_id=id)
    memory = ConversationBufferMemory(
        memory_key="chat_history", llm=llm,max_token_limit=800,chat_memory=message_history, return_messages=True,ai_prefix="A",human_prefix="H"
    )
    return memory

In [105]:
def langchain_agent(memory,tools,llm):
    zero_shot_agent=initialize_agent(
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    #verbose=True,
    max_iteration=1,
    #return_intermediate_steps=True,
    handle_parsing_errors=True,
    memory=memory
)
    return zero_shot_agent

In [109]:
session_id = "4"
#query =  "I'm looking for a session in MGM on 27th with the tag bedrock"
query = "I want know about the session with id SEC234"
prompt_incial = promp_definition()
qa = kendra_ask(kendra_index_id,llm)
tools = define_tools(qa,tools)
memory  = memory_dynamodb(session_id,table_name,llm)
agent = langchain_agent(memory,tools,llm)
agent.agent.llm_chain.prompt=promp_definition()
response = agent(query)

In [110]:
print(response['output'])

The session with id SEC234 is titled "Streamlining security investigations with Amazon Security Lake". It will take place on November 29, 2023 from 21:30 - 22:30 at MGM Grand, Level 3, Chairmans 368. The session will discuss how to gain visibility for comprehensive security investigations and swift incident responses in hybrid environments using Amazon Security Lake. The speakers are Ross Warren, Matt Meck and Andrew Bienert from Seek.


In [7]:
tools

[Tool(name='re-invent-agenda-2023', description='useful when you want search sessions in re:Invent 2023 agenda. This will output documentation in json type. ', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='H', ai_prefix='A', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], ou

In [41]:
def build_memory(table_name, session_id):

    history = DynamoDBChatMessageHistory(table_name=table_name, session_id=session_id)
    # memory_summary  = ConversationSummaryMemory(ai_prefix="A", human_prefix="H", chat_memory=history, llm=llm_instant, memory_key="chat_history", return_messages=True)
    memory = ConversationBufferMemory(ai_prefix="A", human_prefix="H", chat_memory=history, memory_key="chat_history")
    return memory

In [43]:
tools

[Tool(name='re-invent-agenda-2023', description='useful when you want search sessions in re:Invent 2023 agenda. This will output documentation in json type. ', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content="I'm looking for a session were elizabeth fuentes as speaker", additional_kwargs={}, example=False), AIMessage(content=' Based on the documents provided, here is the information for sessions with Elizabeth Fuentes as a speaker:\n\nDocument Title: BOA212_From_Las_Vegas,_NM,_to_Las_Vegas,_NV_Elevating_customer_support.txt\n\nSpeakers: \nfirstName_0: Elizabeth, lastName_0: Fuentes\n\nVenue: MGM Grand \n\nDate: Monday, November 27 2023\n\nSession Title: From Las Vegas, NM, to Las Vegas, NV: Elevating customer support\n\nThis session detail



> Entering new LLMChain chain...
Prompt after formatting:

Human: I'm looking for a session were elizabeth fuentes as speaker
Assistant:  Based on the documents provided, here is the information for sessions with Elizabeth Fuentes as a speaker:

Document Title: BOA212_From_Las_Vegas,_NM,_to_Las_Vegas,_NV_Elevating_customer_support.txt

Speakers: 
firstName_0: Elizabeth, lastName_0: Fuentes

Venue: MGM Grand 

Date: Monday, November 27 2023

Session Title: From Las Vegas, NM, to Las Vegas, NV: Elevating customer support

This session details how AI-driven solutions can transform confounding chat and voice mix-ups into positive customer experiences, with Elizabeth Fuentes listed as a speaker.
Human: Search agenda for "Elizabeth Fuentes" 

Assistant:  Unfortunately there were no documents provided in the <documents> tags, so I do not have any session details to report based on the search term "Elizabeth Fuentes". My role is to truthfully respond based on the information given, and no do

In [45]:
response

{'input': "I'm looking for a session about generative ai",
 'chat_history': '',
 'output': 'Based on the details from the re:Invent 2023 agenda, I would recommend the session titled "Enhance your document workflows with generative AI" taking place on November 27th from 8-9pm at Caesars Forum. This session provides the most information on using generative AI to enhance document workflows, and includes specific venue and timing details. Let me know if you need anything else!'}

In [82]:

def save_item_ddb(table,item):
    response = table.put_item(Item=item)
    return response

In [83]:
table_name_agenda

'QaToAnAgendaStack-TblReInventAgendaB6AA89B7-12IQ4AYMT82CF'

In [84]:
import json
jsonFilePath = "reinvent2023v3.json" 
with open(jsonFilePath) as json_file:
    data = json.load(json_file)

In [85]:
data[1]

{'tag_name': 'Solution/Systems Architect, Role, IT Administrator, Role, Cross Industry, Industry, Amazon Simple Storage Service (S3), Services, Well-Architected Framework, Area of Interest, Financial Services, Industry, Security, Compliance, & Identity, Topic, Monday, Day, MGM Grand, Venue, Storage, Topic, Network & Infrastructure Security, Area of Interest, IT Professional/Technical Manager, Role, Media & Entertainment, Industry, Application Security, Area of Interest, 200 - Intermediate, Level',
 'speakers': 'firstName_0: Sally, lastName_0: Guo, firstName_1: Will, lastName_1: Cavin',
 'thirdpartyid': 'STG213-R',
 'sessionuid': '600089D7-05FE-4AD2-8E6C-142C312A6ED2',
 'title': 'Amazon S3 encryption best practices [REPEAT]',
 'description': '<p>Strong adherence to architecture best practices and proactive controls are the foundations of data security. In this chalk talk, learn best practices for securing your data client-side, server-side, and in transit with Amazon S3 encryption optio